The GPT-2 implementation is adapted from the HuggingFace library: https://huggingface.co/gpt2

In [1]:
import random
import pandas as pd
import wandb

wandb.login()

wandb: Currently logged in as: adityagaydhani (use `wandb login --relogin` to force relogin)


True

In [2]:
handle = 'realDonaldTrump' # Change handle to JoeBiden for training the model on Joe Biden's data

In [3]:
df = pd.read_csv(f'../../data/{handle}.csv')
my_tweets = df['tweet']

In [4]:
def make_dataset(dataset, epochs):
    total_text = '<|endoftext|>'
    tweets = [t for t in dataset]
    for _ in range(epochs):
        random.shuffle(tweets)
        total_text += '<|endoftext|>'.join(tweets) + '<|endoftext|>'
    return total_text

In [5]:
EPOCHS = 4

with open(f'../../data/{handle}_train.txt', 'w') as f:
    data = make_dataset(my_tweets, EPOCHS)
    f.write(data)

## Training

In [6]:
!python ../../scripts/run_language_modeling.py \
    --output_dir=output/$handle \
    --overwrite_output_dir \
    --overwrite_cache \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train --train_data_file=../../data/$handle\_train.txt \
    --logging_steps 20 \
    --per_gpu_train_batch_size 1 \
    --num_train_epochs 1

2020-12-20 16:02:05.302737: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
12/20/2020 16:02:08 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
12/20/2020 16:02:08 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='output/realDonaldTrump', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec20_16-02-08_aditya-XPS-15-9570', logging_first

output/realDonaldTrump
[INFO|integrations.py:371] 2020-12-20 16:02:14,658 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
[WARNING|training_args.py:423] 2020-12-20 16:02:14,659 >> Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: adityagaydhani (use `wandb login --relogin` to force relogin)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using

## Generate new tweets

In [7]:
SENTENCES = ["I think that",
             "I like",
             "I don't like",
             "I want",
             "My dream is"]

In [9]:
seed = random.randint(0, 2**32-1)
examples = []
num_return_sequences = 3

for start in SENTENCES:
    val = !python ../../scripts/run_generation.py \
        --model_type gpt2 \
        --model_name_or_path output/$handle \
        --length 160 \
        --num_return_sequences $num_return_sequences \
        --temperature 1 \
        --p 0.95 \
        --seed $seed \
        --prompt {'"<|endoftext|>' + start + '"'}
    generated = [val[-2*(k+1)] for k in range(num_return_sequences)[::-1]]
    print(f'\nStart of sentence: {start}')
    for i, g in enumerate(generated):
        g = g.replace('<|endoftext|>', '')
        print(f'* Generated #{i+1}: {g}')


Start of sentence: I think that
* Generated #1: I think that giving credit where credit is due, like the likes of Schumer, is unprecedented. These are the lunatics who went to steal our Democracy. Radical Left Democrats with absolutely zero pre-existing conditions!
* Generated #2: I think that everything went according to plan. No one felt compelled to make any concessions. They all got a terrific surprise from our National Security...
* Generated #3: I think that your fault for this election is not your fault, but the fact that he didn't win. Many others have done the same. His small group of fools and pandering, though very good, don’t give a damn, he just ruined your credibility.

Start of sentence: I like
* Generated #1: I like to say I've had a GREAT MAN. He called me "Bored With Brothers," and he was very happy to help me. I am proud to say he is a local hero, and I will never forget him!
* Generated #2: I like everything about my daughter. She loved Parkour, she loves Harney, s